In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, auc

/usr/local/anaconda3/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
#Read the data into dataframe
car_df = pd.read_csv('newCardata.csv')
car_features = pd.read_csv('finalDataPreprocessBinary.csv')
car_label = car_df['FraudFound']
new_syntheticData = pd.read_csv('MwMOTEgeneratedSyntheticData.csv')


OSError: File b'finalDataPreprocessBinary.csv' does not exist

In [30]:
#change the label of the data
labelNo = LabelEncoder()
car_df['FraudFound'] = labelNo.fit_transform(car_df['FraudFound'].astype('str'))
car_label = car_df['FraudFound']
print(car_label.shape)


(15419,)


In [31]:
new_syntheticData.shape

(13573, 52)

In [32]:
#split the data into train and test
print(car_features.shape,car_label.shape)

X_train,X_test,y_train,y_test = train_test_split(car_features,car_label,random_state=3,test_size=0.25)
print('xtrain:',type(X_train))

(15419, 52) (15419,)
xtrain: <class 'pandas.core.frame.DataFrame'>


In [33]:
#model object
model = RandomForestClassifier(n_estimators=100)


In [34]:
#train the model

model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf='deprecated', min_samples_split=2,
            min_weight_fraction_leaf='deprecated', n_estimators=100,
            n_jobs=None, oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [35]:
print('Random forest classifier:')
predicted = model.predict(X_test)
print(type(predicted),predicted.shape,predicted)
print('Accuracy is ',round(accuracy_score(y_test,model.predict(X_test)) * 100,2))


Random forest classifier:
<class 'numpy.ndarray'> (3855,) [0 0 0 ..., 0 0 0]
Accuracy is  93.75


In [36]:
# calculating specifity and sensitivity
# 0  := Negative
# 1 := Positive
cm = confusion_matrix(y_test,predicted)
print("Confusion Matrix:\n",cm)

Confusion Matrix:
 [[3607    2]
 [ 239    7]]


In [37]:
TN, FP, FN, TP = cm.ravel()
print("TN:",TN)
print("FP:",FP)
print("FN:",FN)
print("TP:",TP)

print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
print("Sensitivity:",TP/(TP+FN)*100)
print("Specificity:",TN/(TN+FP)*100)

TN: 3607
FP: 2
FN: 239
TP: 7
Accuracy: 93.7483787289
Sensitivity: 2.84552845528
Specificity: 99.944582987


In [69]:
#MwMote
combine_car_features = pd.read_csv('finalPreprocessBinaryTogether.csv')
combine_car_label = pd.read_csv('label.csv')
print(combine_car_features.shape)
print(combine_car_label.shape)

(28992, 52)
(28992, 1)


In [70]:
#add columns to existing dataframe
combine_car_features['FraudFound']=combine_car_label 

In [71]:
combine_car_features.head()

,Make_0,Make_1,Make_2,Make_3,Make_4,MaritalStatus_0,MaritalStatus_1,PolicyType_0,PolicyType_1,PolicyType_2,...,PoliceReportFiled_Yes,WitnessPresent_No,WitnessPresent_Yes,AgentType_External,AgentType_Internal,RepNumber,Deductible,DriverRating,DayDiff,FraudFound
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0.733333,0.00,0.000000,0.000000,0
1,0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,0,0.933333,0.25,1.000000,0.013587,0
2,0,0,0,0,0,0,1,0,0,0,...,0,1,0,1,0,0.400000,0.25,0.666667,0.035326,0
3,0,0,0,0,1,0,1,0,0,1,...,1,1,0,1,0,0.200000,0.25,0.333333,0.054348,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0.133333,0.25,0.000000,0.040761,0


In [81]:
#shuffle the data.
new_combine_features = combine_car_features.set_index(np.random.permutation(combine_car_features.index))

In [82]:
#now divide the feature and label
new_label = new_combine_features['FraudFound']

#drop the Fraud Found lable
#drop the following attributes
new_combine_features.drop(['FraudFound'],inplace=True,axis=1)
# new_label.shape
# print(new_label)

In [83]:
#train the model
X_train,X_test,y_train,y_test = train_test_split(new_combine_features,new_label,random_state=3,test_size=0.25)

In [84]:
#model object
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train,y_train)

print('Random forest classifier:')
predicted = model.predict(X_test)
print(type(predicted),predicted.shape,predicted)
print('Accuracy is ',round(accuracy_score(y_test,model.predict(X_test)) * 100,2))


Random forest classifier:
<class 'numpy.ndarray'> (7248,) [1 1 0 ..., 0 0 1]
Accuracy is  99.46


In [85]:
# calculating specifity and sensitivity
# 0  := Negative(FraudNotFound)
# 1 := Positive (FraudFound)
cm = confusion_matrix(y_test,predicted)
print("Confusion Matrix:\n",cm)
TN, FP, FN, TP = cm.ravel()
print("TN:",TN)
print("FP:",FP)
print("FN:",FN)
print("TP:",TP)

print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
print("Sensitivity:",TP/(TP+FN)*100)
print("Specificity:",TN/(TN+FP)*100)

Confusion Matrix:
 [[3584   39]
 [   0 3625]]
TN: 3584
FP: 39
FN: 0
TP: 3625
Accuracy: 99.4619205298
Sensitivity: 100.0
Specificity: 98.9235440243


In [87]:
#cross validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn import model_selection
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score


scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

kfold = model_selection.KFold(n_splits=10, random_state=10)
model=RandomForestClassifier(n_estimators=100) 

results = model_selection.cross_validate(estimator=model,X=new_combine_features,y=new_label,cv=kfold,scoring=scoring)

print(np.mean(results['test_accuracy']))
print(np.mean(results['test_precision']))
print(np.mean(results['test_recall']))
print(np.mean(results['test_f1_score']))


0.995619975973
0.940954115606
0.997811358373
0.967524615247
